# Capstone Project - The Battle of Neighborhoods

### Table of contents
___
1. *Problem Statement*
2. *Data description*
3. *Methodology*
4. *Results*
5. *Discussion and Coclusions*

### 1. Problem Statement 
___
I work for a global mulinational located in Milan, IT but due to COVID I left my apartment. Probably, with the vaccination campaigns in place, we will return to the office. Therefore, I'm looking for a new apartment in Milan and I would like to use this opportunity to practice my learnings in Coursera, particularly with Foursquare, in order to answer the relevant questions arisen.
The key question that I would address whitin this project is the following: How can I find a convenient and enjoyable place that fit my interests?
In order to make a comparison and an evaluation of the rental option, below is a list of some "constraints" based on what I am looking for:
- Apartment must be one or two room flat
- Desired location is near a metro station within 1km radius
- Price of rent not exceed €1,200 per month
- Nice to have venues such as gym, food shops and restaurants
Finding an apartment in Milan is always an hard job, especially for one and two rooms flats. Therefore, I believe that this work could be useful first for helping me to find a solution and, in general, for anyone moving to other large city in Italy. 

### 2. Data description 
___
To empirically investigate the research question identified in this study, the following data is required:
- List of Boroughs and Neighborhoods of Milan with their geodata (latitude and longitude)
- List of Subway metro stations in Milan with their address location
- List of apartments for rent in Milan including their price
- List of venues for each Milan neighborhood

To retrieve the list of boroughs and neighborhoods of Milan, the Wikipedia page (URL: https://en.wikipedia.org/wiki/Municipalities_of_Milan) will be used, scraping through the python library BeautifulSoup. To get the list of Subway metro stations in Milan, the CKAN Data API will be used to query the open data provided by the Government. A detailed view of the dataset used in this study can be found at the following [link](https://dati.comune.milano.it/dataset/ds535_atm-fermate-linee-metropolitane). To fetch apartments for rent, including their price, a real estate API will be used. Finally, venues will be collected via the Foursquare API.

#### 2.1 List of Boroughs and Neighborhoods of Milan with their geodata (latitude and longitude)

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

url= "https://en.wikipedia.org/wiki/Municipalities_of_Milan"
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

table_contents = []
table = soup.find("table",{"class":"wikitable sortable"})
table_body = table.find('tbody')

rows = table_body.find_all('tr')

for row in rows:
    cell = {}
    cols = row.findAll('td')
    cols = [ele.text.strip() for ele in cols]
    if cols:
        cell['Borough'] = cols[0]
        cell['BoroughName'] = cols[1]
        cell['Neighborhood'] = cols[5]
        table_contents.append(cell)

milan_neighborhood=pd.DataFrame(table_contents)
milan_neighborhood['Borough'] = milan_neighborhood['Borough'].astype(int)
milan_neighborhood.head()

,Borough,BoroughName,Neighborhood
0,1,Centro storico,"Brera, Centro Storico, Conca del Naviglio, Gua..."
1,2,"Stazione Centrale, Gorla, Turro, Greco, Cresce...","Adriano, Crescenzago, Gorla, Greco, Loreto, Ma..."
2,3,"Città Studi, Lambrate, Porta Venezia","Casoretto, Cimiano, Città Studi, Dosso, Lambra..."
3,4,"Porta Vittoria, Forlanini","Acquabella, Calvairate, Castagnedo, Cavriano, ..."
4,5,"Vigentino, Chiaravalle, Gratosoglio","Basmetto, Cantalupa, Case Nuove, Chiaravalle, ..."


In [2]:
# The code was removed by Watson Studio for sharing.

Load local csv file containing latitude and longitude for each borough


,Borough,Latitude,Longitude
0,1,45.465362,9.188748
1,2,45.492814,9.203981
2,3,45.481547,9.218666
3,4,45.440969,9.217621
4,5,45.445495,9.183412
5,6,45.460022,9.127662
6,7,45.464486,9.088580
7,8,45.502387,9.115245
8,9,45.500830,9.176076


In [3]:
milan_neighborhood = pd.merge(milan_neighborhood, geo_coordinates, on='Borough')
milan_neighborhood.head()

,Borough,BoroughName,Neighborhood,Latitude,Longitude
0,1,Centro storico,"Brera, Centro Storico, Conca del Naviglio, Gua...",45.465362,9.188748
1,2,"Stazione Centrale, Gorla, Turro, Greco, Cresce...","Adriano, Crescenzago, Gorla, Greco, Loreto, Ma...",45.492814,9.203981
2,3,"Città Studi, Lambrate, Porta Venezia","Casoretto, Cimiano, Città Studi, Dosso, Lambra...",45.481547,9.218666
3,4,"Porta Vittoria, Forlanini","Acquabella, Calvairate, Castagnedo, Cavriano, ...",45.440969,9.217621
4,5,"Vigentino, Chiaravalle, Gratosoglio","Basmetto, Cantalupa, Case Nuove, Chiaravalle, ...",45.445495,9.183412
5,6,"Barona, Lorenteggio","Arzaga, Barona, Boffalora, Cascina Bianca, Con...",45.460022,9.127662
6,7,"Baggio, De Angeli, San Siro","Assiano, Baggio, Figino, Fopponino, Forze Arma...",45.464486,9.088580
7,8,"Fiera, Gallaratese, Quarto Oggiaro","Boldinasco, Bullona, Cagnola, Campo dei Fiori,...",45.502387,9.115245
8,9,"Porta Garibaldi, Niguarda","Affori, Bicocca, Bovisa, Bovisasca, Bruzzano, ...",45.500830,9.176076


In [33]:
# Create map of Toronto using latitude and longitude values
# !conda install -c conda-forge folium=0.5.0 --yes
import folium
from branca.element import Figure

latitude = 45.46993590738357
longitude = 9.189059689797839
map_milan = folium.Map(location=[latitude, longitude],
                       zoom_start=12,
                       width='55%',
                       height='55%')

# Add markers to map
for lat, lng, borough, neighborhood in zip(milan_neighborhood['Latitude'], milan_neighborhood['Longitude'], milan_neighborhood['Borough'], milan_neighborhood['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        fill_color='#FFFFFF',
        fill_opacity=0.7,
        parse_html=False).add_to(map_milan)

map_milan

### 3. Methodolody
___


### 4. Results
___

### 5. Discussion and Conclusions
___